In [368]:
import numpy as np
import random

nd = 100 #Number of days
th = 24 #Number of time intervals in a day


#Solar parameters
A = 34 #Area of panel in meter square
seff = 0.16 #Solar panel efficiency in %
MaxSP = 5 #Maximum solar power in KW
NSP = 7 #Number of Solar panels

#Wind Parameters
vc = 3 #Cut-in velocity in m/s
vf = 25 #Cut-off velocity in m/s
vr = 12 #Rated speed in m/s
MaxWP = 5 #Maximum wind power in KW
NWP = 8 #Number of wind turbines


#Battery parameters
BLmax = 36 #Maximum energy level in KWh
BLmin = 7.2 #Minimum energy level in KWh
BLv = 18 #Initial energy level in KWh
Crate = 4 #Maximum charging rate in KWh
Drate = -4 #Maximum discharging rate in KWh

#load[nd][th]
#state[th]
#SI[nd][th]
#V[nd][th]
#Price[th]
NumberofStates = 100
NumberofActions = 3

State = np.zeros([NumberofStates], dtype=float)


action = [-1,0,1] # -1 for discharging, 0 for idle and 1 for charging
    
    



#for i in range(1,24):
#    print(state[i].SOC)

Q_values = np.random.randn(NumberofStates,NumberofActions)
#Q_values = np.zeros([NumberofStates,NumberofActions],dtype=float)
#print(Q_values)

In [369]:
#load = [100][25]
#SI = [100][25]
#V = [100][25]
Price = []
#PV = [100][25]
#Wind = [100][25]
T = 30


Load = [[random.randint(0,5) for x in range(25)] for x in range(100)]
SI = [[random.randint(0,1000) for x in range(25)] for x in range(100)]
V = [[random.randint(5,25) for x in range(25)] for x in range(100)]

for i in range(0,25):
    Price.append(random.randint(3,12))

PV = [[PV_Calc(SI[y][x],seff,A) for x in range(25)] for y in range(100)]
Wind = [[Wind_Calc(V[y][x],vc,vr,vf) for x in range(25)] for y in range(100)]
        
        
        

In [370]:
def PV_Calc(SI,seff,A):
    Ps = NSP*seff*A*SI/1000
    return Ps

def Wind_Calc(v,vc,vr,vf):
    #MaxWP = 5
    if(vf<=v or vc>=v):
        pw = 0
    if(vc<=v and v<=vr):
        pw = (MaxWP*(pow(v,3)-pow(vc,3))/(pow(vr,3)-pow(vc,3)))
    if(vr<= v and v<=vf):
        pw = MaxWP
    return pw*NWP

In [371]:
def state_transition(CurrentState,action):
    deltat = 1
    
    
    if(action == 1):
        CurrentState = CurrentState + deltat*Crate
    if(action == -1):
        CurrentState = CurrentState + deltat*Drate
    if(action == 0):
        CurrentState = CurrentState
    return CurrentState

In [372]:
state_transition(18,1)

22

In [373]:
def Reward(CurrentState,action,NextState,time):
    reward = 0
    if action == 1:
        if NextState >= BLmax:
            reward = -10
        else:
            reward = -State[NextState]*Price[time]
    elif action == 0:
        reward = 0
    elif action == -1:
        if NextState <= BLmin:
            reward = -10
        else:
            reward = State[NextState]*Price[time]
    return reward

In [374]:
def select_epsilon_greedy_action(epsilon, CurrentState):
    result = np.random.uniform()
    action = [-1,1,0]
    if result < epsilon:
        return np.random.choice(action) # Random action (Up,Down,left,right)
    else:
        x = np.argmax(Q_values[CurrentState]) # Greedy action for state
        if(x == 0):
            return -1 #Discharging
        elif(x == 1):
            return 0 #Idle
        elif(x == 2):
            return 1 #Charging


In [375]:
def PGCalc(NextState,day,hour,action):
    u = action*4
    State[NextState] = Load[day][hour]-PV[day][hour]-Wind[day][hour] + u
    

In [376]:
num_episodes = 100
epsilon = 0.2
discount = 0.9 # Discount Factor is set here

for episode in range(num_episodes):
    #initial_state = 0 # STarting state is specified here
    #state = initial_state
    CurrentState = 18
    #print("Episode Begins",episode)
    
    for hour in range(0,25): # Run until the end of the episode
        
        
        #next_state = 37
        # Select action
    
        action = select_epsilon_greedy_action(epsilon, CurrentState)
        NextState = state_transition(CurrentState,action)
        PGCalc(NextState,episode,hour,action)
        reward= Reward(CurrentState,action,NextState,hour) 
           
            
        # Improve Q-values with Bellman Equation
        if hour == 24:
            Q_values[CurrentState][action] = reward
        else:
            Q_values[CurrentState][action] = reward + discount * max(Q_values[NextState])
        CurrentState = NextState
    
    #print("Episode Ends",episode)

print('Q-values are:')
print(Q_values)
action_dict = {0:'Discharge', 1:'Idle',2:'Charge'}
state = 0
for Q_vals in Q_values:
    print('Best action for state {} is {}'.format(state, 
                                             action_dict[np.argmax(Q_vals)]))
    state += 1

Q-values are:
[[ 4.56231088e-01  1.70579469e-01 -6.95909563e-01]
 [-1.00640067e+00  1.17332519e+00 -9.11301799e-02]
 [ 0.00000000e+00  2.88104613e+03 -1.03441766e+01]
 [ 7.89588839e-01  1.02238249e+00 -1.58842604e-01]
 [-1.58080970e+00  1.61524426e+00  2.91970907e-01]
 [-2.31301248e-01 -8.22781427e-01  2.15166257e-01]
 [ 0.00000000e+00  2.98760942e+03  2.58294152e+03]
 [-1.07379572e+00 -5.90393038e-01  9.59475126e-02]
 [ 1.52023331e+00 -4.17007439e-01 -9.44511572e-01]
 [-4.17044611e-01  3.49137360e-01 -1.43237975e-01]
 [ 2.45122381e+03  2.63314483e+03  2.67884848e+03]
 [-4.40586134e-01 -1.45996476e+00  7.73339117e-01]
 [ 1.53868787e-01  1.85291797e-01  1.52333583e+00]
 [-6.21183061e-02 -1.09096113e+00 -1.96717722e+00]
 [ 0.00000000e+00  2.69606954e+03  2.14905150e+03]
 [-6.18105019e-01  7.28305872e-01  3.13365617e-01]
 [-4.06958673e-01 -1.36847491e-01 -2.53263340e-01]
 [-3.22397997e-01 -4.42064728e-01 -1.05017364e+00]
 [ 2.10653202e+03  5.86509120e+02  2.00836179e+03]
 [ 1.49891852e+00

In [377]:
#load = [100][25]
#SI = [100][25]
#V = [100][25]
Price = []
#PV = [100][25]
#Wind = [100][25]
T = 30


Load = [[random.randint(0,5) for x in range(25)] for x in range(100)]
SI = [[random.randint(0,1000) for x in range(25)] for x in range(100)]
V = [[random.randint(5,25) for x in range(25)] for x in range(100)]

for i in range(0,25):
    Price.append(random.randint(3,12))

PV = [[PV_Calc(SI[y][x],seff,A) for x in range(25)] for y in range(100)]
Wind = [[Wind_Calc(V[y][x],vc,vr,vf) for x in range(25)] for y in range(100)]
        
        
        

In [378]:
CurrentState = 65
for hours in range( 25):
    print(CurrentState)
    k = np.argmax(Q_values[CurrentState])
    if k == 0:
        action = -1
    elif k == 1:
        action = 0
    elif k == 2:
        action = 1
    print(action)
    #print(np.argmax(Q_values[CurrentState]))
    print("Best action for  day {} hour is {} ".format( hours, 
                                             action_dict[np.argmax(Q_values[CurrentState])]))
    NextState = state_transition(CurrentState,action)
    CurrentState = NextState
    print(CurrentState)
        
        
        

65
0
Best action for  day 0 hour is Idle 
65
65
0
Best action for  day 1 hour is Idle 
65
65
0
Best action for  day 2 hour is Idle 
65
65
0
Best action for  day 3 hour is Idle 
65
65
0
Best action for  day 4 hour is Idle 
65
65
0
Best action for  day 5 hour is Idle 
65
65
0
Best action for  day 6 hour is Idle 
65
65
0
Best action for  day 7 hour is Idle 
65
65
0
Best action for  day 8 hour is Idle 
65
65
0
Best action for  day 9 hour is Idle 
65
65
0
Best action for  day 10 hour is Idle 
65
65
0
Best action for  day 11 hour is Idle 
65
65
0
Best action for  day 12 hour is Idle 
65
65
0
Best action for  day 13 hour is Idle 
65
65
0
Best action for  day 14 hour is Idle 
65
65
0
Best action for  day 15 hour is Idle 
65
65
0
Best action for  day 16 hour is Idle 
65
65
0
Best action for  day 17 hour is Idle 
65
65
0
Best action for  day 18 hour is Idle 
65
65
0
Best action for  day 19 hour is Idle 
65
65
0
Best action for  day 20 hour is Idle 
65
65
0
Best action for  day 21 hour is Idle 
6

In [379]:
np.argmax(Q_values[20])

1